In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

In [0]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
import numpy as np

class MLP:
  def __init__(self, vocab_size, hidden_size, num_classes):
    self._vocab_size = vocab_size
    self._hidden_size = hidden_size
    self._num_classes = num_classes
  def build_graph(self):
    self._X = tf.placeholder(tf.float32, shape = [None, self._vocab_size])
    self._real_Y = tf.placeholder(tf.int32, shape=[None, ])
    
    weights_1 = tf.get_variable(
        name = 'hidden_input_weights',
        shape = (self._vocab_size, self._hidden_size),
        initializer = tf.random_normal_initializer(seed = 2020)
    )
    biases_1=tf.get_variable(
        name = 'hidden_input_biases',
        shape = (self._hidden_size),
        initializer = tf.random_normal_initializer(seed = 2020)
    )
    weights_2 = tf.get_variable(
        name = 'hidden_output_weights',
        shape = (self._hidden_size, self._num_classes),
        initializer = tf.random_normal_initializer(seed = 2020)
    )
    biases_2 = tf.get_variable(
        name = 'hidden_output_biases',
        shape = (self._num_classes),
        initializer = tf.random_normal_initializer(seed = 2020)
    )
    hiden = tf.sigmoid(tf.matmul(self._X, weights_1) + biases_1)
    logits = tf.matmul(hiden, weights_2) + biases_2
    labels_one_hot = tf.one_hot(indices = self._real_Y, depth = self._num_classes, dtype = tf.float32)
    loss = tf.nn.softmax_cross_entropy_with_logits(labels = labels_one_hot, logits = logits)
    loss = tf.reduce_mean(loss)
    probs = tf.nn.softmax(logits)
    predicted_labels = tf.argmax(probs, axis=1)
    predicted_labels = tf.squeeze(predicted_labels)
    return predicted_labels, loss
  
  def trainer(self, loss, learning_rate):
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    return train_op

class DataReader():
  def __init__(self, data_path, batch_size, vocab_size):
    self._batch_size = batch_size
    with open(data_path, encoding = 'latin1') as f:
      d_lines = f.read().splitlines()

    self._data, self._labels = [], []
    for data_id, line in enumerate(d_lines):
      vector = [0.0 for _ in range(vocab_size)]
      features = line.split('<fff>')
      label,doc_id = int(features[0]), int(features[1])
      for token in features[2].split():
        index, value = int(token.split(':')[0]), float(token.split(':')[1])
        vector[index] = value
      self._data.append(vector)
      self._labels.append(label)
    self._data = np.array(self._data)
    self._labels = np.array(self._labels)
    self._num_epoch = 0
    self._batch_id = 0

  def reset(self):
    self._num_epoch = 0
    self._batch_id = 0

  def next_batch(self):
    start = self._batch_id * self._batch_size
    end = start + self._batch_size
    self._batch_id += 1
    if end + self._batch_size > len(self._data):
      end = len(self._data)
      self._num_epoch += 1
      self._batch_id = 0
      indices = list(range(len(self._data)))
      np.random.seed(2020)
      np.random.shuffle(indices)
      self._data, self._labels = self._data[indices], self._labels[indices]
    return self._data[start:end], self._labels[start:end]

def load_dataset(vocab_size):
  train_data_reader = DataReader(
      data_path = '/content/drive/My Drive/Data_Colab/train_tf_idf_vector.txt',
      batch_size = 50,
      vocab_size = vocab_size
  )
  test_data_reader = DataReader(
      data_path = '/content/drive/My Drive/Data_Colab/test_tf_idf_vector.txt',
      batch_size = 50,
      vocab_size = vocab_size
  )
  return train_data_reader, test_data_reader

def save_parameters(name, value, epoch):
  file_name = name.replace(':', '-colon-') + '-epoch-{}.txt'.format(epoch)
  if len(value.shape) == 1:#is a list
    string_form = ','.join([str(number) for number in value])
  else:
    string_form = '\n'.join([','.join([str(number) for number in value[row]])
                                          for row in range(value.shape[0])])
  with open("/content/drive/My Drive/Data_Colab/saved_paras/"+file_name, "w") as f:
    f.write(string_form)

def restore_parameters(name, epoch):
  file_name = name.replace(':', '-colon-') + '-epoch-{}.txt'.format(epoch)
  with open("/content/drive/My Drive/Data_Colab/saved_paras/"+file_name, "r") as f:
    lines = f.read().splitlines()
  if len(lines) == 1: #is a vector
    value = [float(number) for number in lines[0].split(",")]
  else: #is a matrix
    value = [[float(number) for number in lines[row].split(",")]
             for row in range(len(lines))]
  return value

In [0]:
with open('/content/drive/My Drive/Data_Colab/words_idf.txt',encoding = 'latin1') as f:
  vocab_size = len(f.read().splitlines())
train_data_reader, test_data_reader = load_dataset(vocab_size) 

In [57]:
#create a computation graph
tf.reset_default_graph()
mlp = MLP(vocab_size = vocab_size,
        hidden_size = 50,
        num_classes = 20)
predicted_labels, loss = mlp.build_graph()
train_op = mlp.trainer(loss = loss, learning_rate = 0.1)
train_data_reader.reset()
test_data_reader.reset()
#open a session to run
with tf.Session() as sess:
  step, MAX_STEP = 0,2e3
  sess.run(tf.global_variables_initializer())
  #training
  while step < MAX_STEP:
    train_data, train_labels = train_data_reader.next_batch()
    plabels_eval, loss_eval, _ = sess.run(
        [predicted_labels, loss, train_op],
        feed_dict = {
            mlp._X: train_data,
            mlp._real_Y: train_labels
        }
    )
    step += 1
    print('step: {}, loss: {}'.format(step, loss_eval))

  #save model parameters
  trainable_variables = tf.trainable_variables()
  for variable in trainable_variables:
    save_parameters(
        name = variable.name,
        value = variable.eval(),
        epoch = train_data_reader._num_epoch
    )
  #Danh gia model tren test data
  trainable_variables = tf.trainable_variables()
  for variable in trainable_variables:
    saved_value = restore_parameters(variable.name, train_data_reader._num_epoch)
    assign_op = variable.assign(saved_value)
    sess.run(assign_op)
  num_true_preds = 0
  while True:
    test_data,test_labels = test_data_reader.next_batch()
    test_plabels_eval = sess.run(
        predicted_labels,
        feed_dict = {
            mlp._X: test_data,
            mlp._real_Y: test_labels
        }
    )
    matches = np.equal(test_plabels_eval, test_labels)
    num_true_preds += np.sum(matches.astype(float))

    if test_data_reader._batch_id == 0:
      break

  print('Epoch:', train_data_reader._num_epoch)
  print('Accuracy on test data:', num_true_preds/len(test_data_reader._data))

Epoch: 8
Accuracy on test data: 0.7935475305363782
